# Alpha Algorithm for Process Mining
**Workflow Mining: Discovering Process Models from Event Logs** (_Wil van der Aalst, Ton Weijters, and Laura Maruster_)

In [122]:
%reload_ext autoreload
%autoreload 2
from practical.ProcessMining.group1.task2.alphaminer import AlphaMiner
from practical.ProcessMining.group1.task2.visualization import visualize_full_alpha_miner
import pm4py
import pandas as pd
import csv
import uuid
import os
from datetime import datetime

In [123]:
def import_csv(file_path):
    event_log = pd.read_csv(file_path, sep=';')
    event_log = pm4py.format_dataframe(event_log, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
    return event_log

def import_xes(file_path):
    event_log = pm4py.read_xes(file_path)
    return event_log

def event_log_to_csv(event_log):
    os.makedirs('./tmp_logs', exist_ok=True)
    filename = f'./tmp_logs/event_log_{uuid.uuid4()}.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(["case_id", "activity", "timestamp"])
        for i, activities in enumerate(event_log, start=1):
            for activity in activities:
                timestamp = datetime.now().isoformat()
                writer.writerow([i, activity, timestamp])
    return filename

## Basic Example from the Paper
A simple event log to illustrate the idea behind the algorithm. 

`{ABCD, ACBD, AED}`

In [124]:
simple_log = event_log_to_csv([("a", "b", "c", "d"), ("a", "c", "b", "d"), ("a", "e", "d")])
miner_simple = AlphaMiner(simple_log)
miner_simple.event_log

,case_id,activity,activity_id
0,1,a,0
1,1,b,1
2,1,c,2
3,1,d,3
4,2,a,0
5,2,c,2
6,2,b,1
7,2,d,3
8,3,a,0
9,3,e,4


### 1. Event Log to Footprint Matrix

Steps 1-4 of the Alpha algorithm

In [125]:
print(miner_simple.footprint_matrix())

   a   b   c  d  e
a  #   →   →  #  →
b  ←   #  ||  →  #
c  ←  ||   #  →  #
d  #   ←   ←  #  ←
e  ←   #   #  →  #


In [126]:
footprints = miner_simple.discover_footprints()
for key in footprints.keys():
    print(f"{key}: \n{footprints[key]}\n")

dfg: 
{('a', 'e'), ('c', 'd'), ('b', 'c'), ('c', 'b'), ('a', 'c'), ('b', 'd'), ('a', 'b'), ('e', 'd')}

sequence: 
{('a', 'e'), ('c', 'd'), ('a', 'c'), ('b', 'd'), ('a', 'b'), ('e', 'd')}

parallel: 
{('b', 'c'), ('c', 'b')}

activities: 
{'c', 'a', 'd', 'e', 'b'}

start_activities: 
{'a'}

end_activities: 
{'d'}

min_trace_length: 
3



In [127]:
miner_simple.print_pairs()

Following pairs:
[({'a'}, {'b'}), ({'b'}, {'c'}), ({'a'}, {'e'}), ({'c'}, {'b'}), ({'e'}, {'d'}), ({'c'}, {'d'}), ({'a'}, {'c'}), ({'b'}, {'d'})]

Parallel pairs:
[({'b'}, {'c'}), ({'c'}, {'b'})]

Sequential pairs:
[({'a'}, {'b'}), ({'a'}, {'e'}), ({'e'}, {'d'}), ({'c'}, {'d'}), ({'a'}, {'c'}), ({'b'}, {'d'})]

Not following pairs:
[({'a'}, {'a'}), ({'a'}, {'d'}), ({'b'}, {'b'}), ({'b'}, {'e'}), ({'c'}, {'c'}), ({'c'}, {'e'}), ({'d'}, {'a'}), ({'d'}, {'d'}), ({'e'}, {'b'}), ({'e'}, {'c'}), ({'e'}, {'e'})]

Before pairs:
[({'b'}, {'a'}), ({'c'}, {'a'}), ({'d'}, {'b'}), ({'d'}, {'c'}), ({'d'}, {'e'}), ({'e'}, {'a'})]

Maximal pairs:
[({'a'}, {'e', 'b'}), ({'e', 'b'}, {'d'}), ({'c', 'e'}, {'d'}), ({'a'}, {'c', 'e'})]



### 2. Footprint Matrix to Process Graph
Steps 5-8 of the Alpha algorithm

In [128]:
miner_simple.get_maximal_pairs()

[({'a'}, {'b', 'e'}),
 ({'b', 'e'}, {'d'}),
 ({'c', 'e'}, {'d'}),
 ({'a'}, {'c', 'e'})]

In [129]:
visualize_full_alpha_miner("example_files/common-example.csv", "common-example")

## Extended Example
A slightly more complex example of an event log

In [29]:
import_xes("example_files/running-example.xes")

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

,concept:name,time:timestamp,cost:total,org:resource,@@index,case:concept:name
0,register request,2010-12-30 11:02:00+00:00,50,Pete,14,1
1,examine thoroughly,2010-12-31 10:06:00+00:00,400,Sue,15,1
2,check ticket,2011-01-05 15:12:00+00:00,100,Mike,16,1
3,decide,2011-01-06 11:18:00+00:00,200,Sara,17,1
4,reject request,2011-01-07 14:24:00+00:00,200,Pete,18,1
5,register request,2010-12-30 11:32:00+00:00,50,Mike,9,2
6,check ticket,2010-12-30 12:12:00+00:00,100,Mike,10,2
7,examine casually,2010-12-30 14:16:00+00:00,400,Sean,11,2
8,decide,2011-01-05 11:22:00+00:00,200,Sara,12,2
9,pay compensation,2011-01-08 12:05:00+00:00,200,Ellen,13,2


In [30]:
miner_extended = AlphaMiner("example_files/running-example.xes")
miner_extended.event_log

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

,case_id,activity,activity_id
0,1,register request,5
1,1,examine thoroughly,3
2,1,check ticket,0
3,1,decide,1
4,1,reject request,7
5,2,register request,5
6,2,check ticket,0
7,2,examine casually,2
8,2,decide,1
9,2,pay compensation,4


### 1. Event Log to Footprint Matrix

Steps 1-4 of the Alpha algorithm

In [35]:
miner_extended.footprint_matrix()

,check ticket,decide,examine casually,examine thoroughly,pay compensation,register request,reinitiate request,reject request
check ticket,#,→,||,||,#,←,←,#
decide,←,#,←,←,→,#,→,→
examine casually,||,→,#,#,#,←,#,#
examine thoroughly,||,→,#,#,#,←,←,#
pay compensation,#,←,#,#,#,#,#,#
register request,→,#,→,→,#,#,#,#
reinitiate request,→,←,#,→,#,#,#,#
reject request,#,←,#,#,#,#,#,#


In [36]:
footprints = miner_extended.discover_footprints()
for key in footprints.keys():
    print(f"{key}: \n{footprints[key]}\n")

dfg: 
{('examine casually', 'decide'), ('check ticket', 'examine thoroughly'), ('reinitiate request', 'examine thoroughly'), ('examine casually', 'check ticket'), ('register request', 'examine thoroughly'), ('decide', 'pay compensation'), ('examine thoroughly', 'decide'), ('check ticket', 'decide'), ('decide', 'reinitiate request'), ('reinitiate request', 'check ticket'), ('decide', 'reject request'), ('register request', 'check ticket'), ('examine thoroughly', 'check ticket'), ('check ticket', 'examine casually'), ('register request', 'examine casually')}

sequence: 
{('examine casually', 'decide'), ('reinitiate request', 'examine thoroughly'), ('register request', 'examine thoroughly'), ('decide', 'pay compensation'), ('examine thoroughly', 'decide'), ('check ticket', 'decide'), ('decide', 'reinitiate request'), ('reinitiate request', 'check ticket'), ('decide', 'reject request'), ('register request', 'check ticket'), ('register request', 'examine casually')}

parallel: 
{('examine t

In [37]:
miner_extended.print_pairs()

Following pairs:
[({'check ticket'}, {'decide'}), ({'examine casually'}, {'decide'}), ({'examine thoroughly'}, {'decide'}), ({'check ticket'}, {'examine thoroughly'}), ({'examine casually'}, {'check ticket'}), ({'decide'}, {'pay compensation'}), ({'examine thoroughly'}, {'check ticket'}), ({'decide'}, {'reject request'}), ({'check ticket'}, {'examine casually'}), ({'register request'}, {'check ticket'}), ({'reinitiate request'}, {'check ticket'}), ({'register request'}, {'examine thoroughly'}), ({'decide'}, {'reinitiate request'}), ({'reinitiate request'}, {'examine thoroughly'}), ({'register request'}, {'examine casually'})]

Parallel pairs:
[({'check ticket'}, {'examine thoroughly'}), ({'examine casually'}, {'check ticket'}), ({'examine thoroughly'}, {'check ticket'}), ({'check ticket'}, {'examine casually'})]

Sequential pairs:
[({'check ticket'}, {'decide'}), ({'examine casually'}, {'decide'}), ({'examine thoroughly'}, {'decide'}), ({'decide'}, {'pay compensation'}), ({'decide'}, {

### 2. Footprint Matrix to Process Graph
Steps 5-8 of the Alpha algorithm

In [39]:
miner_extended.get_maximal_pairs()

[({'decide'}, {'pay compensation', 'reinitiate request'}),
 ({'check ticket'}, {'decide'}),
 ({'decide'}, {'reinitiate request', 'reject request'}),
 ({'register request', 'reinitiate request'}, {'check ticket'}),
 ({'register request', 'reinitiate request'}, {'examine thoroughly'}),
 ({'decide'}, {'pay compensation', 'reject request'}),
 ({'register request'}, {'examine casually', 'examine thoroughly'}),
 ({'examine casually', 'examine thoroughly'}, {'decide'})]

In [40]:
visualize_full_alpha_miner("example_files/running-example.xes", "extended-example")

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

## Limitations

The algorithm has several limitations that are important to consider:

- Implicit places (places that are redundant)
- Loops of length 1 and 2
- Non-local dependencies
- Representational bias (cannot discover transitions with duplicate or invisible labels)
- Discovered model does not need to be sound
- Noise (infrequent behaviour/incorrectly recorded events)
- Incompleteness (missing events)

### Implicit Places
The algorithm does not discover implicit places, which are places that are redundant in the model. For example, in the following event log, the Alpha algorithm would discover the following model:

`{ACEG, AECG, BDFG, BFDG}`

In [130]:
implicit_log = event_log_to_csv([("a", "c", "e", "g"), ("a", "e", "c", "g"), ("b", "d", "f", "g"), ("b", "f", "d", "g")])
miner_implicit = AlphaMiner(implicit_log)
miner_implicit.event_log

,case_id,activity,activity_id
0,1,a,0
1,1,c,2
2,1,e,4
3,1,g,6
4,2,a,0
5,2,e,4
6,2,c,2
7,2,g,6
8,3,b,1
9,3,d,3


In [132]:
print(miner_implicit.footprint_matrix())

   a  b   c   d   e   f  g
a  #  #   →   #   →   #  #
b  #  #   #   →   #   →  #
c  ←  #   #   #  ||   #  →
d  #  ←   #   #   #  ||  →
e  ←  #  ||   #   #   #  →
f  #  ←   #  ||   #   #  →
g  #  #   ←   ←   ←   ←  #


In [133]:
miner_implicit.print_pairs()

Following pairs:
[({'c'}, {'e'}), ({'a'}, {'e'}), ({'b'}, {'f'}), ({'e'}, {'g'}), ({'e'}, {'c'}), ({'a'}, {'c'}), ({'c'}, {'g'}), ({'f'}, {'g'}), ({'d'}, {'g'}), ({'f'}, {'d'}), ({'b'}, {'d'}), ({'d'}, {'f'})]

Parallel pairs:
[({'c'}, {'e'}), ({'e'}, {'c'}), ({'f'}, {'d'}), ({'d'}, {'f'})]

Sequential pairs:
[({'a'}, {'e'}), ({'b'}, {'f'}), ({'e'}, {'g'}), ({'a'}, {'c'}), ({'c'}, {'g'}), ({'f'}, {'g'}), ({'d'}, {'g'}), ({'b'}, {'d'})]

Not following pairs:
[({'a'}, {'a'}), ({'a'}, {'b'}), ({'a'}, {'d'}), ({'a'}, {'f'}), ({'a'}, {'g'}), ({'b'}, {'a'}), ({'b'}, {'b'}), ({'b'}, {'c'}), ({'b'}, {'e'}), ({'b'}, {'g'}), ({'c'}, {'b'}), ({'c'}, {'c'}), ({'c'}, {'d'}), ({'c'}, {'f'}), ({'d'}, {'a'}), ({'d'}, {'c'}), ({'d'}, {'d'}), ({'d'}, {'e'}), ({'e'}, {'b'}), ({'e'}, {'d'}), ({'e'}, {'e'}), ({'e'}, {'f'}), ({'f'}, {'a'}), ({'f'}, {'c'}), ({'f'}, {'e'}), ({'f'}, {'f'}), ({'g'}, {'a'}), ({'g'}, {'b'}), ({'g'}, {'g'})]

Before pairs:
[({'c'}, {'a'}), ({'d'}, {'b'}), ({'e'}, {'a'}), ({'f'}, {

In [134]:
# TODO: visualizing the model

### Loops of Length 1 and 2

The algorithm does not discover loops of length 1 and 2. For example, in the following event log, the Alpha algorithm would discover the following model:

`{AC, ABC, ABBC, ABBBBC}`
`{ABD, ABCBD, ABCBCBD}`

## Real Example

In [44]:
miner_real = AlphaMiner("sorted_session_data.csv")

In [45]:
miner_real.discover_footprints()

{'dfg': {('10-stappen', 'WDO211959'),
  ('50plus', 'mijn_berichten'),
  ('50plus', 'top20-vacatures'),
  ('50plus', 'vacatures_bij_mijn_cv'),
  ('PORTAL.wwsec_app_priv.logout', 'home'),
  ('Portal (ignore_request_page)', 'taken'),
  ('WDO211559', 'WDO211559'),
  ('WDO211559', 'inschrijven'),
  ('WDO211559', 'mijn_berichten'),
  ('WDO211559', 'mijn_cv'),
  ('WDO211559', 'mijn_tips'),
  ('WDO211559', 'mijn_werkmap'),
  ('WDO211559', 'taken'),
  ('WDO211559', 'vacatures_bij_mijn_cv'),
  ('WDO211559', 'vacatures_zoeken'),
  ('WDO211559', 'wdo_010304'),
  ('WDO211559', 'werkmap'),
  ('WDO211959', 'top20-vacatures'),
  ('aanvragen-ww', 'PORTAL.wwsec_app_priv.logout'),
  ('aanvragen-ww', 'aanvragen-ww'),
  ('aanvragen-ww', 'home'),
  ('aanvragen-ww', 'mijn_werkmap'),
  ('aanvragen-ww', 'overzicht'),
  ('bevestigingspagina', 'mijn_cv'),
  ('bijstandsuitkering', 'inschrijven'),
  ('certificaat-ervaring', 'wdo_014546'),
  ('eigen-bedrijf', '10-stappen'),
  ('email_uw_vraag', 'email_uw_vraag'),
 

In [46]:
miner_real.get_maximal_pairs()

[({'solliciteren', 'wdo_014546'}, {'online-training'}),
 ({'mijn_tips', 'vacatures_bij_mijn_cv'}, {'mijn_bewijs_van_inschrijving'}),
 ({'inschrijven'}, {'solliciteren', 'wdo211818'}),
 ({'inschrijven'}, {'mijn_documenten', 'uwvsanlig-webfont.eot'}),
 ({'inschrijven'}, {'gegevens', 'mijn_sollicitaties'}),
 ({'foutopgetreden.html', 'gegevens'}, {'home'}),
 ({'inschrijven'}, {'geen-digid', 'mijn_online_trainingen'}),
 ({'Portal (ignore_request_page)', 'WDO211559'}, {'taken'}),
 ({'mijn_online_trainingen', 'mijn_tips'}, {'mijn_bewijs_van_inschrijving'}),
 ({'inschrijven'}, {'foutopgetreden.html', 'mijn_online_trainingen'}),
 ({'inschrijven'}, {'gegevens', 'uwvsanlig-webfont.eot'}),
 ({'solliciteren'}, {'online-training', 'taken'}),
 ({'mijn_bewijs_van_inschrijving', 'wdo_010304'}, {'home'}),
 ({'inschrijven'}, {'mijn_online_trainingen', 'vragenlijst-uwv'}),
 ({'inschrijven'}, {'aanvragen-ww', 'mijn_sollicitaties'}),
 ({'top20-vacatures', 'werkmap'}, {'mijn_berichten'}),
 ({'Portal (ignore_

In [47]:
miner_real.footprint_matrix()

,10-stappen,50plus,PORTAL.wwsbr_javascript.page_js,PORTAL.wwsec_app_priv.logout,Portal (ignore_request_page),WDO211559,WDO211959,aanvragen-ww,bevestigingspagina,bijstandsuitkering,...,wdo_010304,wdo_014546,werkmap,werkplan_lightbox,wijziging_doorgeven,ww,zoekAantalIndicatief,zoekBeroep,zoekBeroepenlijst,zoeken
10-stappen,#,#,#,#,#,#,→,#,#,#,...,#,#,#,#,#,#,#,#,#,#
50plus,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
PORTAL.wwsbr_javascript.page_js,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
PORTAL.wwsec_app_priv.logout,#,#,#,#,#,#,#,←,#,#,...,#,#,#,#,#,#,#,#,#,#
Portal (ignore_request_page),#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ww,#,#,#,#,#,#,#,→,#,#,...,#,#,#,#,#,#,#,#,#,#
zoekAantalIndicatief,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,||,←,#,#
zoekBeroep,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,→,||,#,#
zoekBeroepenlijst,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,||,#
